In [1]:
import sys
sys.path.insert(0, '../lib')  # noqa
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
import pdb
from sklearn.metrics import *
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import itertools
import csv
from lasagne import layers
from lasagne.updates import nesterov_momentum
from lasagne.nonlinearities import softmax
from nolearn.lasagne import NeuralNet
from greedy_order import *


C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\lenovo\Anaconda3\lib\site-packages\theano\tensor\signal\downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


ModuleNotFoundError: No module named 'nolearn'

In [2]:
import os
ipath='F:/TREC-RTS2016/25617/'
jpath='F:/TREC-RTS2016/25617/0.1JM_rank/'
dpath='F:/TREC-RTS2016/25617/1000Dirichlet/'
bpath='F:/TREC-RTS2016/25617/bm25_list/'


In [3]:
#ip=['MB320','MB256']
ip=['MB226','MB229','MB230','MB239','MB254','MB256','MB258','MB265','MB267','MB276','MB286','MB319','MB320','MB332','MB351','MB358','MB361','MB362','MB363','MB365','MB371','MB377','MB381','MB382','MB391','MB392','MB409','MB410','MB414','MB419','MB420','MB425','MB431','MB436','MB438','MB440','RTS1','RTS10','RTS13','RTS14','RTS19','RTS2','RTS21','RTS24','RTS25','RTS27','RTS28','RTS31','RTS32','RTS35','RTS36','RTS37','RTS4','RTS43','RTS5','RTS6']


In [ ]:

train_df=pd.DataFrame()
#train_df.columns=['tweet_id','JM','Diri','BM25','Class']

fout=open('C:/Users/lenovo/AnacondaProjects/learning-to-rank-master/experiments/train_deatil.txt','a+')
for i,p in enumerate(ip):
    qrels=pd.read_csv('C:/Users/lenovo/Google Drive/PHD share folder/paper/trec experiment/analysis/qrels.txt',sep=' ')
    qn=qrels[qrels['profile'] == p]
    qn=qn[['profile','tweet_id','relevance']]
    os.chdir(ipath)
    #print (p)
    os.chdir(jpath+p)
    jdf=pd.read_csv(p+'.txt',header=None,sep=',',names=['date', 'profile','q0','tweet_id','rank','score','text'])
    jdf=jdf.drop_duplicates(subset=['tweet_id','profile'])
    jdf=jdf[['profile','tweet_id','score']]
    jdf.head()

    os.chdir(dpath+p)
    ddf=pd.read_csv(p+'.txt',header=None,sep=',',names=['date', 'profile','q0','tweet_id','rank','score','text'])
    ddf=ddf.drop_duplicates(subset=['tweet_id','profile'])

    ddf=ddf[['profile','tweet_id','score']]
    ddf.head()
    
    
    
    os.chdir(bpath+p)
    bdf=pd.read_csv(p+'.txt',header=None,sep=',',names=['date', 'profile','q0','tweet_id','rank','score','text'])
    bdf=bdf.drop_duplicates(subset=['tweet_id','profile'])

    bdf=bdf[['profile','tweet_id','score']]
        
    #n_df =  pd.DataFrame(list(set(ddf['tweet_id'].tolist()) & set(jdf['tweet_id'].tolist())))
    
    jd_df=pd.merge(jdf,ddf,how='inner',on=['tweet_id','profile'])
    bdj_df=pd.merge(jd_df,bdf,how='inner',on=['tweet_id','profile'])
    #bdj_df.info()
    qbdj_df=pd.merge(bdj_df,qn,how='inner',on=['tweet_id','profile'])
    qbdj_df.columns=['profile','tweet_id','JM','Diri','BM25','Class']
    qbdj_df['JM']=round(qbdj_df['JM'],2)                
    qbdj_df['Diri']=round(qbdj_df['Diri'],2)                
    qbdj_df['BM25']=round(qbdj_df['BM25'],2)
    
    qbdj_df['Class'] = qbdj_df.Class.apply(lambda x: x if not pd.isnull(x) else 0)
    qbdj_df['Class']=qbdj_df.Class.astype(int)
    qbdj_df.info()
    
    instr = 'profile'+ "  " + str(len(qn['tweet_id']))+"  "+str(len(qbdj_df['tweet_id'])) +'\n'
    fout.write(instr)
                          
    qbdj_df.to_csv('C:/Users\lenovo\AnacondaProjects\learning-to-rank-master\experiments/sample.txt',encoding='UTF-8',index=False,header=None,mode='a')
fout.close()


    

In [4]:
#ip2017=['RTS101']
path2017='F:/TREC2017/Analysis/Ranklist/'
jpath2017='F:/TREC2017/Analysis/Ranklist/jm112/'
dpath2017='F:/TREC2017/Analysis/Ranklist/diri/'
bpath2017='F:/TREC2017/Analysis/Ranklist/bm25/'
import os
import pandas as pd

In [5]:
ip2017=[]
with open('F:/TREC2017/Analysis/tid95-2.txt','r') as fin:
    line= fin.readlines()
    for l in line:
        l=l.strip().split(' ')
        #tpn=l[0]
        tpid=l[1]
        ip2017.append(tpid)      
#print len(ip2017)
#ip2017.remove('RTS63')
#ip2017.remove('RTS164')
#
#ip2017.remove('RTS182')


In [6]:
len(ip2017)

95

In [23]:
fout=open('C:/Users/lenovo/AnacondaProjects/learning-to-rank-master/experiments/test_deatil.txt','a+')
qrels=pd.read_csv('F:/TREC2017\Analysis\Qrel/rts2017-batch-qrels.txt',sep=' ',names=['profile','q0','tweet_id','relevance'])

for i,p in enumerate(ip2017):
    
    qn=qrels[qrels['profile'] == p]
    qn=qn[['profile','tweet_id','relevance']]
    
    os.chdir(path2017)
    #print (p)
    os.chdir(jpath2017+p)
    jdf=pd.read_csv(p+'.txt',header=None,sep=',',names=['date', 'profile','q0','tweet_id','rank','score','text'])
    jdf=jdf.drop_duplicates(subset=['tweet_id','profile'])
    jdf=jdf[['profile','tweet_id','score']]
    jdf.head()

    os.chdir(dpath2017+p)
    ddf=pd.read_csv(p+'.txt',header=None,sep=',',names=['date', 'profile','q0','tweet_id','rank','score','text'])
    ddf=ddf.drop_duplicates(subset=['tweet_id','profile'])

    ddf=ddf[['profile','tweet_id','score']]
    #ddf.head()
    
    
    
    os.chdir(bpath2017+p)
    bdf=pd.read_csv(p+'.txt',header=None,sep=',',names=['date', 'profile','q0','tweet_id','rank','score','text'])
    bdf=bdf.drop_duplicates(subset=['tweet_id','profile'])

    bdf=bdf[['profile','tweet_id','score']]
        
    #n_df =  pd.DataFrame(list(set(ddf['tweet_id'].tolist()) & set(jdf['tweet_id'].tolist())))
    
    jd_df=pd.merge(jdf,ddf,how='inner',on=['tweet_id','profile'])
    bdj_df=pd.merge(jd_df,bdf,how='inner',on=['tweet_id','profile'])
    #bdj_df.info()
    bdj_df.columns=['profile','tweet_id','JM','Diri','BM25']
    bdj_df['JM']=round(bdj_df['JM'],2)                
    bdj_df['Diri']=round(bdj_df['Diri'],2)                
    bdj_df['BM25']=round(bdj_df['BM25'],2)
    
    #bdj_df.info()
    
    instr = p+ "  " + str(len(qn['tweet_id']))+"  "+str(len(bdj_df['tweet_id'])) +'\n'
    fout.write(instr)
                          
    #bdj_df.to_csv('C:/Users\lenovo\AnacondaProjects\learning-to-rank-master\experiments/test.txt',encoding='UTF-8',index=False,header=None,mode='a')
    
    qbdj_df=pd.merge(bdj_df,qn,how='inner',on=['tweet_id','profile'])
    qbdj_df.columns=['profile','tweet_id','JM','Diri','BM25','Class']
    qbdj_df['JM']=round(qbdj_df['JM'],2)                
    qbdj_df['Diri']=round(qbdj_df['Diri'],2)                
    qbdj_df['BM25']=round(qbdj_df['BM25'],2)
    
    qbdj_df['Class'] = qbdj_df.Class.apply(lambda x: x if not pd.isnull(x) else 0)
    qbdj_df['Class']=qbdj_df.Class.astype(int)
    #qbdj_df.info()
    
    qbdj_df.to_csv('C:/Users\lenovo\AnacondaProjects\learning-to-rank-master\experiments/gtest.txt',encoding='UTF-8',index=False,header=None,mode='a')

    
fout.close()  


 


In [20]:
#jdf=pd.read_csv(p+'.txt',header=None,sep=',',names=['date', 'profile','q0','tweet_id','rank','score','text'])
fout.close()
p

'RTS182'

In [8]:
'''
X_train=pd.read_csv('C:/Users\lenovo\AnacondaProjects\learning-to-rank-master\experiments/sample.txt',sep=','
                    ,header=None,names=['profile','tweet_id','JM','Diri','BM25','Class'])
X_train.head()

scaler = StandardScaler()
X_train['JM'] = scaler.fit_transform(X_train[['JM']])
X_train['Diri'] = scaler.fit_transform(X_train[['Diri']])
X_train['BM25'] = scaler.fit_transform(X_train[['BM25']])       
X_train.head()

X_trainall['Class']=np.where(X_trainall['Class'] ==2,1, X_trainall['Class'])
'''
from sklearn.preprocessing import StandardScaler

X_trainall=pd.read_csv('C:/Users\lenovo\AnacondaProjects\learning-to-rank-master\experiments/sample.txt',sep=','
                    ,header=None,names=['profile','tweet_id','JM','Diri','BM25','Class'])
#X_trainall.head()

scaler = StandardScaler()
X_trainall['JM'] = scaler.fit_transform(X_trainall[['JM']])
X_trainall['Diri'] = scaler.fit_transform(X_trainall[['Diri']])
X_trainall['BM25'] = scaler.fit_transform(X_trainall[['BM25']]) 

X_trainall['Class']=np.where(X_trainall['Class'] ==2,1, X_trainall['Class'])
X_trainall.head()

print(X_trainall['Class'].value_counts())
X_train0=X_trainall[X_trainall['Class']==0]
X_train1=X_trainall[X_trainall['Class']==1]
#X_train0.info()
#X_train1.info()
X_train0=X_train0.sample(3029)
#X_train0.info()
X_trainws=pd.concat([X_train0,X_train1])
X_train = X_trainws.reindex(np.random.permutation(X_trainws.index))
X_train.info()
#X_trainws.info()
#X_train.head()



0    34799
1     3029
Name: Class, dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 6058 entries, 22440 to 36329
Data columns (total 6 columns):
profile     6058 non-null object
tweet_id    6058 non-null int64
JM          6058 non-null float64
Diri        6058 non-null float64
BM25        6058 non-null float64
Class       6058 non-null int64
dtypes: float64(3), int64(2), object(1)
memory usage: 331.3+ KB


In [9]:
X_test=pd.read_csv('C:/Users\lenovo\AnacondaProjects\learning-to-rank-master\experiments/gtest.txt',sep=','
                    ,header=None,names=['profile','tweet_id','JM','Diri','BM25','Class'])
X_test.head()

scaler = StandardScaler()
X_test['JM'] = scaler.fit_transform(X_test[['JM']])
X_test['Diri'] = scaler.fit_transform(X_test[['Diri']])
X_test['BM25'] = scaler.fit_transform(X_test[['BM25']])       
X_test.head()


,profile,tweet_id,JM,Diri,BM25,Class
0,RTS47,891302728198230020,3.050570,2.407051,3.571360,1
1,RTS47,891348035069935616,2.778155,1.815867,2.271354,2
2,RTS47,891359347099320320,2.778155,1.815867,2.271354,2
3,RTS47,891391584524279809,2.778155,1.815867,2.271354,2
4,RTS47,891299170077769728,2.621369,1.806629,1.665027,2


In [27]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6058 entries, 17565 to 22657
Data columns (total 6 columns):
profile     6058 non-null object
tweet_id    6058 non-null int64
JM          6058 non-null float64
Diri        6058 non-null float64
BM25        6058 non-null float64
Class       6058 non-null int64
dtypes: float64(3), int64(2), object(1)
memory usage: 331.3+ KB


In [10]:
X_train['Class']=np.where(X_train['Class'] ==2,1, X_train['Class'])
X_test['Class']=np.where(X_test['Class'] ==-2,0, X_test['Class'])
X_test['Class']=np.where(X_test['Class'] ==2,1, X_test['Class'])

In [12]:
X_train['Class'].value_counts()
X_test['Class'].value_counts()

0    51755
1     5561
Name: Class, dtype: int64

In [13]:
X_train1=X_train[['JM','Diri','BM25','Class']]
print(X_train1.groupby('Class').mean())
y_train=X_train['Class']
xx_train=X_train1[['JM','Diri','BM25']]


             JM      Diri      BM25
Class                              
0     -0.091851 -0.099656 -0.088536
1      0.942935  1.002133  0.886593


In [14]:
X_test1=X_test[['JM','Diri','BM25','Class']]
print(X_test1.groupby('Class').mean())
xx_test=X_test1[['JM','Diri','BM25']]
y_test=X_test1['Class']


             JM      Diri      BM25
Class                              
0     -0.087395 -0.094564 -0.076164
1      0.813362  0.880082  0.708838


In [15]:
X_test.shape

(57316, 6)

In [16]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
# train the model using X_train_dtm
%time logreg.fit(xx_train, y_train)

Wall time: 29 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [17]:
y_pred_test = logreg.predict(xx_test)

In [18]:
 #calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_test)

0.75331495568427664

In [19]:
# print the confusion matrix
metrics.confusion_matrix(y_test, y_pred_test)

array([[39920, 11835],
       [ 2304,  3257]])

In [20]:
print (metrics.precision_score(y_test, y_pred_test))
print(metrics.recall_score(y_test, y_pred_test))
print (metrics.f1_score(y_test, y_pred_test))
print(metrics.classification_report(y_test, y_pred_test,target_names=['NR','R']))
#y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
#y_pred_prob
print(metrics.roc_auc_score(y_test, y_pred_test))

0.215809700504
0.585686027693
0.315402120757
             precision    recall  f1-score   support

         NR       0.95      0.77      0.85     51755
          R       0.22      0.59      0.32      5561

avg / total       0.87      0.75      0.80     57316

0.678506234791


In [21]:
X_test['Class_pred']=y_pred_test
run_df=pd.read_csv('D:/datas/2017output/result/est-res2017/diri_linreg_est_20170.1 23.0_6.0.txt',sep=' '
                    ,header=None,names=['date','profile','q0','tweet_id','rank','score','tag'])
run_df.info()
#X_test.to_csv('C:/Users\lenovo\AnacondaProjects\learning-to-rank-master\experiments/Test_res.txt',index=False,header=None,sep=' ')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13018 entries, 0 to 13017
Data columns (total 7 columns):
date        13018 non-null int64
profile     13018 non-null object
q0          13018 non-null object
tweet_id    13018 non-null int64
rank        13018 non-null int64
score       13018 non-null float64
tag         13018 non-null object
dtypes: float64(1), int64(3), object(3)
memory usage: 712.0+ KB


In [22]:
X_test['Class_pred'].value_counts()

rundft=run_df[['tweet_id']]
rundft.head()
X_testtid = X_test[X_test['Class_pred']==0]
X_testtid=X_testtid[['tweet_id']]
#X_testtid.info()

ctid = pd.DataFrame(list(set(rundft['tweet_id'].tolist()) & set(X_testtid['tweet_id'].tolist())))
ctid.info()
ctid.columns=['tweet_id']
#ltid=ctid[0].values.tolist()
#len(ltid)
#for tid in ltid:
 #   run_df.ix[~run_df['tweet_id']==tid]
deltid=pd.DataFrame(list(set(rundft['tweet_id'].tolist()) - set(ctid['tweet_id'].tolist())))
deltid.info()
deltid.columns=['tweet_id']

newrun_df=pd.merge(run_df,deltid,how='inner',on=['tweet_id'])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 959 entries, 0 to 958
Data columns (total 1 columns):
0    959 non-null int64
dtypes: int64(1)
memory usage: 7.6 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11822 entries, 0 to 11821
Data columns (total 1 columns):
0    11822 non-null int64
dtypes: int64(1)
memory usage: 92.4 KB


In [23]:
run_df.info()
newrun_df.info()
newrun_df.head()
newrun_df.to_csv('D:/datas/2017output/result/lr-res/diripointlr.txt',sep=' ',header=None,index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13018 entries, 0 to 13017
Data columns (total 7 columns):
date        13018 non-null int64
profile     13018 non-null object
q0          13018 non-null object
tweet_id    13018 non-null int64
rank        13018 non-null int64
score       13018 non-null float64
tag         13018 non-null object
dtypes: float64(1), int64(3), object(3)
memory usage: 712.0+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 12026 entries, 0 to 12025
Data columns (total 7 columns):
date        12026 non-null int64
profile     12026 non-null object
q0          12026 non-null object
tweet_id    12026 non-null int64
rank        12026 non-null int64
score       12026 non-null float64
tag         12026 non-null object
dtypes: float64(1), int64(3), object(3)
memory usage: 751.6+ KB


In [21]:
from sklearn.neural_network import MLPClassifier
#per = Perceptron()


In [22]:
mlp = MLPClassifier(activation='logistic',solver='sgd',hidden_layer_sizes=(6,3,1))
mlp.fit(xx_train,y_train)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(6, 3, 1), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='sgd', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [23]:
y_pred_ntest = mlp.predict(xx_test)


In [24]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,y_pred_ntest))

[[  0 585]
 [  0 108]]


In [25]:
print (metrics.precision_score(y_test, y_pred_ntest))
print(metrics.recall_score(y_test, y_pred_ntest))
print (metrics.f1_score(y_test, y_pred_ntest))
print(metrics.classification_report(y_test, y_pred_ntest,target_names=['NR','R']))
#y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
#y_pred_prob
print(metrics.roc_auc_score(y_test, y_pred_ntest))

0.155844155844
1.0
0.269662921348
             precision    recall  f1-score   support

         NR       0.00      0.00      0.00       585
          R       0.16      1.00      0.27       108

avg / total       0.02      0.16      0.04       693

0.5


C:\Users\lenovo\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [26]:
X_test['Class_pred'].value_counts()

0    553
1    140
Name: Class_pred, dtype: int64

In [ ]:
x